In [1]:
import pandas as pd
import pyodbc
username = 'vigrose'
password_aims = 'DQA8K@T7pXEqEt'
password_edw = 'Hufflepuff10946'

In [2]:
scraped = pd.read_csv('../../Data/SRS/SRS_Scrape_2021-01-13.csv')
scraped_new = pd.read_csv('../../Data/SRS/SRS_Scrape_2021-03-19.csv')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (27,28,29,30,31,32,33,34,35,36,41,42,43,44,65,66,67,68,69,70,71,72,73,74,79,80,81,82,84,85,86,87,88,89,90,91,92,93,98,99,100,101) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (8,22,23,24,25,27,28,29,30,31,32,33,34,35,36,41,42,43,44,65,66,67,68,69,70,71,72,73,74,79,80,81,82,84,85,86,87,88,89,90,91,92,93,98,99,100,101) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
s = "DSN=aims_prod; UID={}; PWD={}".format(username, password_aims)
informix = pyodbc.connect(s)

In [24]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [ ]:
query = \
    f'''
    SELECT 
    E.KEY_TYPE_VAL AS AAMC_ID,
    S.STUD_ID,
    S.SCHOOL_ID,
    S.EDU_STS,
    S.GRAD_YR,
    S.EDU_PREF_CD,
    S.DEGREE_CD
    FROM informix.ENTITY_KEY_ET E, informix.SCHOOL_ATT_ST S
    WHERE
    E.ACTIVE_IND = 'Y'
    AND
    E.KEY_TYPE = 'AMC'
    AND
    E.ENTITY_ID = S.STUD_ID
    AND
    S.END_DT IS NULL
    ;
    '''
query_2 = \
    f'''
    SELECT 
    S.STUD_ID,
    S.SCHOOL_ID,
    S.EDU_STS,
    S.GRAD_YR,
    S.EDU_PREF_CD,
    S.DEGREE_CD,
    P.ENTITY_ID
    P.GENDER
    P.BIRTH_DT
    P.BIRTH_STATE_CD
    P.ETHNICITY
    FROM
    informix.SCHOOL_ATT_ST S, informix.ENTITY_KEY_ET E, informix.PERSON_NAME_ET N, informix.PERSON_ET P
    WHERE
    S.END_DT IS NULL
    ;
    '''

In [90]:
query_3 = \
    f'''
    SELECT 
    P.ENTITY_ID,
    P.GENDER,
    P.BIRTH_DT,
    P.BIRTH_STATE_CD,
    P.ETHNICITY,
    P.BIRTH_CITY_CD,
    N.FIRST_NM,
    N.MIDDLE_NM,
    N.LAST_NM,
    S.STUD_ID,
    S.SCHOOL_ID,
    S.EDU_STS,
    S.GRAD_YR,
    S.EDU_PREF_CD,
    S.DEGREE_CD
    FROM
    informix.PERSON_NAME_ET N, informix.PERSON_ET P, informix.SCHOOL_ATT_ST S
    WHERE
    P.ENTITY_ID = N.ENTITY_ID
    AND
    P.CURR_PERSON_TYPE = 'S'
    AND
    N.END_DT IS NULL
    AND
    N.NAME_TYPE='LN'
    AND
    P.ENTITY_ID = S.STUD_ID
    ;
    '''
all_students = pd.read_sql(con=informix, sql=query_3)
all_students.head()

,entity_id,gender,birth_dt,birth_state_cd,ethnicity,birth_city_cd,first_nm,middle_nm,last_nm,stud_id,school_id,edu_sts,grad_yr,edu_pref_cd,degree_cd
0,6015941,M,1965-07-06,MS,,SARDIS,Phillip,Newton,Lambert,6015941,14796570,WTH,1996,None,MD
1,6016994,M,1970-11-25,MS,,JACKSON,Timothy,Craig,Howard,6016994,14796570,WTH,1996,None,MD
2,6017111,M,1962-07-17,NY,,S OZONE PARK,Louis,James,Brunoforte,6017111,14796570,WTH,1996,None,MD
3,6018983,F,1972-05-06,MS,,COLUMBUS,Latanya,Denise,Mc Knight,6018983,14796570,WTH,1998,None,MD
4,6020621,F,1973-09-20,MS,,OXFORD,April,Renee,Wise,6020621,14796570,DSM,1999,None,MD


In [27]:
query_5 = \
    f'''
    SELECT 
    E.KEY_TYPE_VAL AS AAMC_ID,
    E.ENTITY_ID
    FROM informix.ENTITY_KEY_ET E
    WHERE
    E.KEY_TYPE = 'AMC'
    AND
    E.ACTIVE_IND = 'Y'
    ;
    '''
active_aamc = pd.read_sql(con=informix, sql=query_5)
active_aamc.head()

,aamc_id,entity_id
0,10592157,4627372
1,10592155,4675056
2,10169352,2745752
3,10592159,4627177
4,10592162,4527363


In [339]:
query_6 = \
    f'''
    SELECT 
    E.KEY_TYPE_VAL AS ME,
    E.ENTITY_ID
    FROM informix.ENTITY_KEY_ET E, informix.PERSON_ET P
    WHERE
    E.KEY_TYPE = 'ME'
    AND
    E.ACTIVE_IND = 'Y'
    AND
    P.ENTITY_ID = E.ENTITY_ID
    ;
    '''
me_entity = pd.read_sql(con=informix, sql=query_6)
me_entity.head()

,me,entity_id
0,00102730780,1262088
1,00102730810,1262127
2,00102730844,1262166
3,00102730879,1262205
4,00102730909,1262244


In [109]:
query_7 = \
    f'''
    SELECT 
    E.ENTITY_ID,
    E.CATEGORY_CODE,
    E.BEGIN_DT,
    E.END_DT
    FROM informix.ENTITY_CAT_CT E
    WHERE
    E.CATEGORY_CODE = 'STU-AFFIL'
    AND
    E.END_DT IS NULL
    ;
    '''
STU_AFFIL = pd.read_sql(con=informix, sql=query_7)
STU_AFFIL.head()

,entity_id,category_code,begin_dt,end_dt
0,51256930,STU-AFFIL,2018-11-06,2018-11-10
1,51257424,STU-AFFIL,2018-11-06,2018-11-10
2,51257541,STU-AFFIL,2018-11-06,2018-11-10
3,50145768,STU-AFFIL,2018-08-28,2018-11-09
4,50145781,STU-AFFIL,2018-08-28,2018-11-09


In [344]:
scraped_new = scraped_new.rename(columns = {'AAMC ID': 'aamc_id'})
active_aamc['aamc_id'] = active_aamc.aamc_id.astype(int)

In [31]:
active_aamc['aamc_id'] = active_aamc.aamc_id.astype(int)

In [91]:
students = all_students

In [110]:
all_students = pd.merge(students, STU_AFFIL, on='entity_id', how='left')
all_students = all_students.fillna('None')

In [345]:
found = pd.merge(scraped_new, active_aamc, on='aamc_id')
found_info = pd.merge(found, all_students, on='entity_id')
found_info = found_info.fillna('None')
stu_affil = found_info[(found_info.category_code=='STU-AFFIL ')]

In [351]:
found_students = found_info[(found_info.category_code!='STU-AFFIL ')]

In [427]:
missing = scraped_new[scraped_new.aamc_id.isin(found_info.aamc_id)==False]
missing['last_nm'] = [x.split(', ')[0] for x in missing.Name]
missing['first_middle'] = [x.split(', ')[1] for x in missing.Name]
missing['birth_state_cd'] = [us_state_abbrev[x] if x in us_state_abbrev.keys() else x for x in missing['Birth State']]
missing['birth'] = pd.to_datetime(missing['Date of Birth'])
with_birth = all_students[all_students.birth_dt!='None']
with_birth['birth'] = pd.to_datetime([str(x).replace(' ','') for x in with_birth.birth_dt])

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a c

In [125]:
def remove_spaces(name):
    name = name.replace('  ','')
    if len(name)>1:
        if name[-1] == ' ':
            name = name[:-1]
    return(name)

In [354]:
first_middles = []
for row in all_students.itertuples():
    first = remove_spaces(row.first_nm)
    if row.middle_nm != 'None':
        middle = ' '+ remove_spaces(row.middle_nm)
    else:
        middle = ''
    first_middle = first + middle
    print(first_middle)
    first_middles.append(first_middle)
all_students['first_middle'] = first_middles

Phillip Newton
Timothy Craig
Louis James
Latanya Denise
April Renee
Patricia Angela
Lori Ann
Kathleen C
Pernell Victor
Michael Nelson
Charlotte M
Renee Adele
Holly Kathryn
William Paul
Kimberly Yvette
James H
Reginald D
Ryan Beatty
Michael Robert
Camilla Marie
Laury Le Ann
Janice Ruth
Frank Edwin
Robt Ray
Chas Luther
Craig Slaton
John Victor
Mary Elias
Thomas 
Jeffrey Mason
Tracy Ann
Craig Scott
Aaron Demetrius
Tu Xuan
Stephen Earl
Travis Sasha
David Glenn
Larry 
Jason Frederick
James Maxwell
Catherine J
Steve Cheng Fong
Constance 
Julio Pablo
Christopher Nwaeze
Robt Jos
Jaydie Lynne
Robert Harald
Todd Andrew
Diane Jean
Deborah Kathalene
Nancy Jean
Eliot T
Joe 
Amy Janine
Kristina Theresa
Stephanie D
Abel G
Syngfu Frank
Alexandra 
Mark Dearl
Jerry Babb
Benjamin Monte
Wm Eugene
Yuesin 
Robt Jos
Jesse Dee
Chandra Danette
Peter Seongsoo
Kristen Enid
William Stephen
John Frederick
Kenneth Kyle
Harold Rudd
Thanh-Quang 
Elizabeth Ann
Sheldon Bernard
Mark Thompson
Christopher B
Kelley Victori

Todd Bennett
Sonya Alyse
G 
David Michael
Sharon Anita
Barbara Ann
Christopher S
Gustavo Bruno
Elizabeth H
Giselle 
Gregory Hratch
Laszlo 
Michael 
Jennifer Elaine
Jack Chas
William Thomas
Alison Lee
Tyrone Lawrence
Justin Rene
Lori Denice
Erin Michelle
Brenda Gayle
Larrie Dale
Jason Daniel
Susan Mann
Heather Ashley
Jennifer Mei
Emill 
Richard Mark
Michael Dean
David Michael
Johann Ulysses
Jose Nicolas
Rhonda Denise
Talitha Poulson
Wayne Drake
Mark Edward
Elisabeth Ann
Catherine Joy
Yolanda Marie
Jose F
James Howard
Andrea Gracecoriet
John Anthony
Gary Rolf
Marc Michael
Leon Daniel
Laurie A
Fredrick Warren
David Yandell
Joel Keith
Stephen Sungmin
Jonathan Mark
Theresa Louise
Henry Nkeruka
Tehmina 
Tehmina 
Satya Srinivas
Thos Jerome
Susan Hayden
Maria Victoria
Christopher A
Danny Earl
Stephen Sean
Jeffrey Peter
Jeffrey I
Moe Dibo
Marilyn 
Albert 
Eileen Dolores
Sam Alan
Danl Jack
Eileen T
Sajad Syed
Lee Ann Avril
Amrit Kaur
Cynthia Denise
Hope Rene
Michael Jos
John Foster
Matthew Lane


Mary Paszek
Donald James
Douglass Edward
Marie L
Vaughn Erwin
Kenny Neil
Wm H
Booker R
Don 
Laide Adebayo
Ali Bey
Chas Loran
Dallas Timothy
Richard Alan
Christian Shunbok
Kimberly Kay
Terence Alexander
Susan Lorelei
Richard Deane
Amir M
Charles Enzio
Charles Enzio
Hany 
Raul Patrick
Mark Nelson
Charlie Anthony
Barbara Maria
Luis Jose
David Anthony
Douglas David
Neil A
Randall A
Guy Richard
Kathy J
Steven Arthur
Todd P
Margaret Jonghee
Cindy Cheeshen
Todd David
Michael John
Patrick Richard
Steven John
Thierry M
Eric Michael
Sarah 
Steven Leroy
Theodore Alexander
Jerilyn Denise
Elizabeth Ann
Timothy James
Lynne Evelyn
Sandra Ann
Ralph Joseph
Fiona Arleen
Leslie Kathryn
Patricia Jean
Samaria 
Joseph Francis
Jean Kristina
Elizabeth M
Lee Edwin
Diann C
Albert Mark
Gautam 
Johanna Lannon
Kimuli Selina
Melinda Lane
Clent Rene
Jennifer Marie
Jean Del
Dianne 
Robert Ernest
Clifton Bernard
Courtney Duval
Cutia Warland
Cutia Warland
Craig Alan
Terry Jay
Derrick F
D 
Ricardo Antonio
Jose L
Victor 

Sidney L
Robt G
Cathy Ann
Julio Ceasar
Wm Clifford
Chas Henry
Manuel Jose
Paul J
Francis 
Joel 
Robt Honore
Jonathan 
Duncan Gregory
Blake Ashley I
Eluid H
Ollie Vernon
Ruben 
Efrem 
Nina Maria
Steven Benjamin
Gary Michael
Frederick Otis
Eduardo Modesto
Todd William
James Harris
Stephen Douglas
Innocent I
Warren M
Loan Thiphuong
Mireille 
Dahlia Eve
Neil Saml
Frank Cha
Warwrick S
Ami 
Deepak 
Margery Ellen
Andrew Michael
George Maleko
Jorge Armando
Noemy 
James L
Michael Gordon
Richard Burke
Robyn Anne
Gaylyn Ann
Elizabeth Ann
Anne Elizabeth
Jennifer Jenhuey
Faith E
Marcia 
Robbie Joseph
Yasmeen Ilana
Scott Adrian
Salim Nizarali
Claude Anthony
Gillian Ursula
Peter James
Sarah Lee
I Jeanette
Marla L
Rico Cortez
Chad Everett
Todd W
Beatrice Lee
Matthew Larry
Susan Elaine
Linda Bowman
James A
Melissa Ann
Robin Elizabeth
Megan Marie
Andrew Dow
Suzanne 
Sandra Patricia
James Lee
Maria K
Nikki Jo
Zaira R
Sheymar 
Yolanda Denise
Mark Hamilton
Clifton Bentley
Olga Tamara
Laura Jane
Chas Larry


Humberto Chile
Kevin Chas
John Chas
Geo Michael
Barry Latrent
Victoria M
Rhonda Eugenia
Natalie A
Donna Fichtner
Susan Danielsen
John Andrew
Mary Victoria
Marshall R
Ernest Curt
Jos Wm
Kim E
Konrad Ernest
John Francis
Victoria Lee
Lavaughn J
Kenneth Lamar
Joni Marie
Tari Lynn
Andrea 
Nakul 
Julie Patricia
Diana Rachel
Matthew Gary
Michael Jay
Bernard Lawrence
Vashti Liley
Michael Adam
Hai Nam
Kenneth Christian
Walter Jos
Nadine M
John 
Fabienne Yves
Carol Ann
Susan E
Anna 
Tyrone 
Elise Madeleine
Maria Liesl
Crystal Shantrai
Marc Elliot
David Menghao
Matthew 
Jeffrey David
Susan Victoria
Alexander K
Brian Jos
Daniel Michael
Raafia 
Andrea Jeanne
Michael Bernard
James John
Saeed F
Patrick John
Giao Vinh
Lane Mack
Vera Jean
Bret Allyn
Herbert John
Laurie 
Laura Ann
Roxanne 
Bokeun 
Ashley Victoria
Melissa Beth
Kathleen Joan
Norman David
Norman David
Marilyn Sue
Yvette Diana
Ronald Mark
John F
Elena 
Paul Darin
Mary Susan
Michelle H
Eileen Patrice
Kimberly Rene
Jean Marie
Angelia Dawn
Mar

...

Nicholas Steven
Nicholas Joseph
Nicholas Connor
Nicholas Robert
Nicholas James
Nicholas Christian Von
Nicholas John
Nicholas Patrick
Nicholas Evan
Nicholas Scott
Nicholas Joshua
Nicholas Marius
Nicholas Ryan
Nicholas Alexius
Nicholas 
Nicholas 
Nicholas Ryan
Nicholas Scott
Nicholas Monroe
Nicholas Naim
Nicholas Michael
Nicholas William
Nicholas Franz
Nicholas J
Nicholas Alexander
Nicholas Antonio
Nicholas James
Nicholas Francis
Nicholas James
Nicholas Frederick
Nicholas John
Nicholas Cheng
Nicholas Luke
Nicholas Victor
Nick Hossein
Nickalous Anthony
Nicki 
Nickolas A
Nickolas Andrew
Nicky 
Nicola 
Nicola Elise
Nicolaas Everett
Nicolai Patrick
Nicolas Alexander
Nicolas Xavier
Nicolas John
Nicolas Kevin
Nicolas Matheo
Nicolas Edward
Nicolas James
Nicolas Andres
Nicolas 
Nicole 
Nicole Elyse
Nicole Deborah
Nicole Frances
Nicole Anna
Nicole Alexandra
Nicole Lauren Brobin
Nicole Elaine
Nicole Theresa
Nicole Michaela
Nicole Renae
Nicole 
Nicole Michelle
Nicole Lee
Nicole Elizabeth
Nicole Mar

Samantha Paige
Samantha Morgan
Samantha Louise
Samantha Mariko
Samantha Chineye
Samantha 
Samantha Sonya
Samantha Lynn
Samantha Maria
Samantha Anne
Samantha Lauren Ryva
Samantha Emily
Samantha Melanie
Samantha Margaret
Samantha Roselyn
Samantha Marie
Samantha Alexis
Samantha Regan
Samantha Ann
Samantha Allison
Samantha Yao
Samantha Skye
Samantha 
Samantha Marie
Samantha Brooke
Samantha Grace
Samantha Paige
Samantha Justina
Samantha 
Samar Shannon
Samar Shanbhag
Samara 
Samarth 
Samarth Chetan
Samay Dipak
Sameen Yasar-Bin
Sameena Shariq
Sameer Ramzan
Sameer Iqbal
Sameera Sara
Samer Ibrahim
Samer 
Samhita Darshini
Sami Anas
Sami I.
Sami 
Sami Milan
Sami 
Samina 
Samir 
Samir 
Samir Suhas
Samir Alejandro
Samira 
Sammer Ahmad
Sampreeti 
Sampson Tom
Samson Gerald
Samson David
Samson Kahsu
Samual 
Samuel John
Samuel 
Samuel Chidera Ezechukwu
Samuel Itzhak
Samuel 
Samuel Dennis
Samuel Bennett
Samuel David
Samuel Nicholas
Samuel Ari
Samuel William
Samuel Earl
Samuel 
Samuel Ajayi
Samuel David


Tyler Brian
Tyler Aaron
Tyler Renee
Tyler James
Tyler John
Tyler Brennan
Tyler Gregory
Tyler Nicole
Tyler David
Tyler James
Tyler Matthew
Tyler Anthony
Tyler Joseph
Tyler James
Tyler Brent
Tyler Brendt
Tyler 
Tyler Alexander
Tyler Anthony
Tyler 
Tyler 
Tyler Jacob
Tyler Preston Stanley
Tyler Natee
Tyler 
Tyler Robert
Tyler Andrew
Tyler Wen
Tyler Paul
Tyler Abram
Tyler Jacob
Tyler Alden
Tyler James
Tyler Scott
Tyler Alan
Tyler Elizabeth
Tyler Alexander
Tyler Adam
Tyler Allen
Tyler Cole
Tyler Christian
Tyler Walter
Tyler James
Tyler 
Tyler James
Tyler Nicole
Tyler Joseph
Tyler Robert
Tyler Allen
Tyler Joshua
Tyler Kenneth
Tyler Wade
Tylor Joseph
Tyona Dominique
Tyra Lynn
Tyric Trey
Tyson K
Tyson 
Tzu Ning 
Uchechi 
Uchechukwu Obialom-Ezenwa
Uchechukwu 
Uchechukwu Chinoso
Udeyvir Singh
Udit R
Uduak-Obong Iniabasi
Ugoji 
Uh Jin 
Ujwal 
Ulises Jose
Ulysse George
Uma 
Uma Diwakar
Umair 
Umair Fareed
Umair Mohammad
Umair Mahmood
Umasai 
Umer Syed
Umu Hawa
Un Bi 
Una Emma
Uniaza 
Urvashi 
Urvi

Nicholas 
Alyssa 
Shelby 
Dylan 
Marisa 
Yasmine 
Tracy 
Mary 
Jena 
Michael 
Sathyadev 
Edward 
Sasha 
Hannah 
Vinh 
Bethany 
Radhika 
Yuliya 
Almalea B
Dominique 
Christina 
Darcy 
Emily 
Kevin 
Amanda 
Andrew Kenneth
Aditya 
Ladan 
Hayley 
Amrit 
Bianca Lynn
Laura Elizabeth
Keeyon 
Caroline Frances
Mariana Denisita
Teresa Elizabeth
Luciana Ava
Alyssa Lenove
Joanna 
Joseph William
Julia Rita
Rachel Elizabeth
Sara C
Tal 
Molly Paige
Nicholas 
Madison 
Nicole 
Joel 
Samantha 
Cassidy 
Abdelrahman 
Gabriella Elisabeth
Kelsey 
Alena 
Yolanda 
Tsaina 
Morgen 
Heather 
Karolina 
Joshua 
Shawn 
Rachel 
Sydney 
Harsh 
Jonathan 
Gianfranco 
Alexis Marie
Ben 
Michael 
Leo 
James 
Frank 
Bailee 
Jennifer 
Rachel 
Dania 
William 
Grace 
Carla 
Samuel 
Rugile 
Susan Hong
Rachel 
Morgan Lyn
Thomas 
Ednord 
Angie 
Harshal 
Fuji 
Deena 
Keri 
Anthony 
Rohith 
Lonnie 
Christopher 
Washington 
Mykia Simone
Letori 
Ashanta 
Schyler 
Nia 
Odiase 
Junie 
Jessica 
Amanda 
Taha 
Steven 
Anjelo 
Andrew 
Mar

Christine 
James Scott
Alex 
Daryl 
Avidor 
Samantha 
Jaswin 
Leon 
Pamela 
Brett 
Logan 
Auryana 
Hashani 
Christie 
Kory Caitlan
Patrick Ryan
Tylor 
Victoria Lynslee
Aaron 
Craig 
Ekei 
Nicolle Anne 
Charlie 
Jonathan 
Christine 
Annika 
Brittani 
Ming 
Tasha 
Clement 
Kavir 
Rumin 
Andrew 
Emily 
Rachael 
Carter 
Claudia 
Michael 
Ryan 
Julia Ann
Megha 
Gabriella 
Kenzie 
Steven 
Farha 
Samiyyah 
McKenna 
Grant 
Allen 
Edward 
Mia 
Augusta 
James 
Sasha 
Taylor 
Garrett 
Vinod 
Bailey 
Obaid 
Rebecca 
Claire 
Rachel 
Michael 
Priya 
Kareem 
Stephanie 
Megan 
Sonia 
Patrick 
Briana 
Shannon 
Jonathan 
Melissa 
Donald 
Siddharth 
Humza 
Amanda 
Ava 
Rebecca 
Saheb 
Tejaswini 
Sean 
Sarah 
John 
Jovesh 
Alexander 
Kyle 
Pratik 
Vipsa 
Taylor 
Amrutha 
Raymond 
Autumn 
Mitchell 
Sara 
Britney 
Jeanne 
Shawn 
Dustin 
Rosemary 
Pranay 
Alma 
Nishitha 
Sanisha 
Emma 
Ayaz 
Cade 
Dominique 
Jacob 
Jordi 
Yalda 
Alyssa 
Hira 
Cassidy 
Saloni 
Cuong 
Elizabeth 
Robert 
Kyle 
Sameer 
Hayden 
L

In [129]:
all_students['last_nm'] = [remove_spaces(x) for x in all_students.last_nm]

In [428]:
bop = pd.merge(with_birth, missing, left_on = ['birth','last_nm', 'first_middle','gender', 'birth_state_cd'], right_on = ['birth','last_nm', 'first_middle','Sex', 'birth_state_cd'])

In [429]:
boop = bop.drop_duplicates(['entity_id', 'aamc_id'])
boop = boop[boop.end_dt=='None']
match = boop.drop_duplicates('aamc_id',keep=False).sort_values('grad_yr').drop_duplicates('aamc_id', keep='last')

In [430]:
still_missing = missing[missing.aamc_id.isin(matched.aamc_id)==False]
stateless_match = pd.merge(with_birth, still_missing, left_on = ['birth','last_nm', 'first_middle','gender'], right_on = ['birth','last_nm', 'first_middle','Sex'])

In [359]:
# add_match = pd.merge(person_students_3, still_missing, left_on = ['birth','last_nm', 'gender'], right_on = ['birth','last_nm', 'Sex']).drop_duplicates('aamc_id')
# boopa = still_missing[still_missing.aamc_id.isin(add_match.aamc_id)==False]
# no_birth_match = pd.merge(without_birth, boopa, left_on = ['first_middle','last_nm', 'gender'], right_on = ['first_middle','last_nm', 'Sex'])[['Name','first_nm','last_nm','birth_state_cd_x','birth_state_cd_y']]

In [431]:
foreign = missing[missing['Birth State']=='Foreign']
foreign_match = pd.merge(with_birth, foreign, left_on = ['birth','last_nm', 'first_middle','gender'], right_on = ['birth','last_nm', 'first_middle','Sex'])
foreign_match = foreign_match[foreign_match.end_dt=='None']

In [432]:
still_missing = missing[missing.aamc_id.isin(matched.aamc_id)==False]
stateless_match = pd.merge(with_birth, still_missing, left_on = ['birth','last_nm', 'first_middle','gender'], right_on = ['birth','last_nm', 'first_middle','Sex'])

In [433]:
stateless_match = stateless_match[stateless_match.end_dt=='None']

In [434]:
weird_matches = pd.merge(with_birth, still_missing, left_on = ['first_middle','last_nm', 'gender','birth'], right_on = ['first_middle','last_nm', 'Sex', 'birth']).sort_values(['end_dt', 'grad_yr']).drop_duplicates('aamc_id', keep='last')

In [435]:
no_first = pd.merge(with_birth, still_missing, left_on = ['birth','last_nm', 'gender'], right_on = ['birth','last_nm', 'Sex'])
keeps =[]
for row in no_first.itertuples():
    keep=False
    if row.first_nm.replace(' ','') in row.Name:
        keep=True
    keeps.append(keep)
no_first['KEEP']=keeps
no_first = no_first[no_first.KEEP==True].drop(columns=['KEEP'])

In [436]:
matched = pd.concat([match, foreign_match, stateless_match, weird_matches, no_first])
matched = matched.drop_duplicates(['aamc_id','entity_id'])
matched = matched.sort_values('grad_yr').drop_duplicates('aamc_id', keep='last')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [223]:
# pure = matched.drop_duplicates('aamc_id', keep=False)
# duplicates = matched[matched.aamc_id.isin(pure.aamc_id)==False]

In [255]:
status = pd.read_excel('../../Data/SRS/Status_Crosswalk.xlsx')
schools = pd.read_excel("../../Data/SRS/US_Schools_Crosswalk_AMA_AAMC.xlsx")

In [367]:
founded = found_students[found_students['Curr Exp Grad Date']!='None']
unfounded = found_students[found_students['Curr Exp Grad Date']=='None']
founded['CURRENT_GRAD_YEAR'] = [x.year for x in pd.to_datetime(founded['Curr Exp Grad Date'])]
unfounded['CURRENT_GRAD_YEAR'] = 'None'
found_students = pd.concat([founded, unfounded])
grad_match = []
for row in found_students.itertuples():
    if row.CURRENT_GRAD_YEAR != row.grad_yr:
        grad_match.append(False)
    else:
        grad_match.append(True)
found_students['grad_match'] = grad_match
grad_mismatch = found_students[found_students.grad_match == False]
grad_mismatch = pd.merge(grad_mismatch, status, left_on='Curr Status', right_on='STATUS_DESC', how='left')
grad_mismatch = pd.merge(grad_mismatch, schools, on='INST_ID', how='left')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [368]:
student_status = pd.merge(found_students, status, left_on='Curr Status', right_on='STATUS_DESC', how='left')
student_status = student_status.rename(columns={'AMA Status':'ama_status'})

In [370]:
status_match = []
for row in student_status.itertuples():
    if row.ama_status != row.edu_sts:
        status_match.append(False)
    else:
        status_match.append(True)
student_status['status_match'] = status_match
status_mismatch = student_status[student_status.status_match == False]
status_mismatch = pd.merge(status_mismatch, schools, on='INST_ID', how='left')
status_update = status_mismatch[status_mismatch.aamc_id.isin(grad_year.aamc_id)==False]

In [ ]:
not_found[['aamc_id',
'Birth Country',
'Birth County',
'Birth State',
'Citizen Country',
'Curr Class Level',
'Curr Class Level Eff Date',
'Curr Exp Grad Date',
'Curr Med School Campus',
'Curr Status',
'Curr Status AcadYear',
'Curr Status Date',
'Current Degree Program',
'Date of Birth',
'Email',
'INST_ID',
'Legal Country',
'Legal County',
'Legal State',
'Matriculation Degree Program',
'Name',
'Preferred USMLE ID',
'RaceEthnicity',
'Sex',
'Visa Desc',
'stud_id',
'school_id',
'edu_sts',
'grad_yr',
'edu_pref_cd',
'degree_cd',
'CURRENT_GRAD_YEAR',
'STATUS_CD',
'STATUS_DESC',
'STATUS_LBL',
'AMA Status',
'AMA Status Reason',
'AMA Rule',
'INST_TYPE',
'INST_NAME',
'AMA_ID']]

In [371]:
grad_year = grad_mismatch[['aamc_id',
  'stud_id',
  'grad_yr',
  'CURRENT_GRAD_YEAR',
'school_id',
'edu_sts',
'edu_pref_cd',
'degree_cd',
'STATUS_CD',
'STATUS_DESC',
'STATUS_LBL',
'AMA Status',
'AMA Status Reason',
'AMA Rule',
'INST_TYPE_CD',
'INST_NAME_120',
'AMA_ID',
'Birth Country',
'Birth County',
'Birth State',
'Citizen Country',
'Curr Class Level',
'Curr Class Level Eff Date',
'Curr Exp Grad Date',
'Curr Med School Campus',
'Curr Status',
'Curr Status AcadYear',
'Curr Status Date',
'Current Degree Program',
'Date of Birth',
'Email',
'INST_ID',
'Legal Country',
'Legal County',
'Legal State',
'Matriculation Degree Program',
'Name',
'Preferred USMLE ID',
'RaceEthnicity',
'Sex',
'Visa Desc']]
#.to_csv('../../Data/SRS/SRS_Graduation_Year_Mismatch_2021-01-13.csv', index=False)

In [481]:
still_missing = pd.merge(still_missing, schools, on='INST_ID', how='left')

In [482]:
still_missing = still_missing[['INST_ID',
'INST_TYPE_CD',
'INST_NAME_120',
'AMA_ID','aamc_id',
'Birth Country',
'Birth County',
'Birth State',
'Citizen Country',
'Curr Class Level',
'Curr Class Level Eff Date',
'Curr Exp Grad Date',
'Curr Med School Campus',
'Curr Status',
'Curr Status AcadYear',
'Curr Status Date',
'Current Degree Program',
'Date of Birth',
'Email',
'Legal Country',
'Legal County',
'Legal State',
'Matriculation Degree Program',
'Name',
'Preferred USMLE ID',
'RaceEthnicity',
'Sex',
'Visa Desc']]
# .to_csv('../../Data/SRS/SRS_Missing_2021-01-13.csv', index=False)

In [372]:
status_update = status_update[['aamc_id',
  'stud_id',
  'grad_yr',
  'CURRENT_GRAD_YEAR',
'school_id',
'edu_sts',
'edu_pref_cd',
'degree_cd',
'STATUS_CD',
'STATUS_DESC',
'STATUS_LBL',
'ama_status',
'AMA Status Reason',
'AMA Rule',
'INST_TYPE_CD',
'INST_NAME_120',
'AMA_ID',
'Birth Country',
'Birth County',
'Birth State',
'Citizen Country',
'Curr Class Level',
'Curr Class Level Eff Date',
'Curr Exp Grad Date',
'Curr Med School Campus',
'Curr Status',
'Curr Status AcadYear',
'Curr Status Date',
'Current Degree Program',
'Date of Birth',
'Email',
'INST_ID',
'Legal Country',
'Legal County',
'Legal State',
'Matriculation Degree Program',
'Name',
'Preferred USMLE ID',
'RaceEthnicity',
'Sex',
'Visa Desc']]
#.to_csv('../../Data/SRS/SRS_Status_Mismatch_2021-01-13.csv', index=False)

In [268]:
rulz = pd.read_excel('../../Data/SRS/Processing _Rules.xlsx')

In [375]:
yall = pd.concat([grad_year, status_update]).drop_duplicates('aamc_id')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [460]:
stats = pd.merge(rulz, status_update, left_on = ['Curr Status', 'edu_status'], right_on =['Curr Status', 'edu_sts'])

In [461]:
manual = stats[(stats['PROCESS?']=='N')&(stats['NEW EDU_STS/ACTION']=='Flag for Manual Review')]

In [468]:
auto = stats[(stats['PROCESS?']=='Y')][['aamc_id','NEW EDU_STS/ACTION','NEW STS REASON','NEW CATEGORY CODE','Curr Status Date','Curr Exp Grad Date']]

In [379]:
all_other_sts = status_update[(status_update.aamc_id.isin(auto.aamc_id)==False)&(status_update.aamc_id.isin(manual.aamc_id)==False)]

In [380]:
other_grad = grad_year[(grad_year.aamc_id.isin(auto.aamc_id)==False)&(grad_year.aamc_id.isin(manual.aamc_id)==False)]

In [455]:
stats[(stats['PROCESS?']=='N')].groupby('NEW EDU_STS/ACTION').count()

,STATUS_CD_x,Curr Status,edu_status,PROCESS?,NEW STS REASON,NEW CATEGORY CODE,aamc_id,stud_id,grad_yr,CURRENT_GRAD_YEAR,...,INST_ID,Legal Country,Legal County,Legal State,Matriculation Degree Program,Name,Preferred USMLE ID,RaceEthnicity,Sex,Visa Desc
NEW EDU_STS/ACTION,,,,,,,,,,,,,,,,,,,,,
Flag for Manual Review,1408,1408,1408,1408,0,0,1408,1408,1408,1408,...,1408,1408,1408,1408,1408,1408,1408,1408,1408,1408


In [462]:
other_sts = status_update[status_update.aamc_id.isin(stats.aamc_id)==False]

In [465]:
unprocessed = stats[(stats['PROCESS?']=='N')&(stats['NEW EDU_STS/ACTION']!='Flag for Manual Review')]

In [457]:
len(status_update)

4762

In [467]:
manual = pd.concat([manual, other_sts])

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [472]:
# scraped
#found
# matched
# missing
# found_students
# stu_affil
# grad_year
# status_update
# manual
# auto
# new
print(
f'''
{"{:,}".format(len(scraped_new))} student records in AAMC-SRS scrape
{"{:,}".format(len(found_info))} ({round(len(found_info)/len(scraped_new)*100, 2)}%) records matched to AIMS on aamc_id
{"{:,}".format(len(matched))} ({round(len(matched)/len(scraped_new)*100, 2)}%) records found via matching process 
{"{:,}".format(len(me_matched))} ({round(len(me_matched)/len(scraped_new)*100, 2)}%) MEs appended via matching process
{"{:,}".format(len(still_missing))} ({round(len(still_missing)/len(scraped_new)*100, 2)}%) scraped records missing from our database

----

Of the records that matched on aamc_id
{"{:,}".format(len(stu_affil))} ({round(len(stu_affil)/len(found)*100, 2)}%) records are student affiliates
{"{:,}".format(len(found_students))} ({round(len(found_students)/len(found)*100, 2)}%) records are not student affiliates
    
----
Of the records that are not student affiliates
{"{:,}".format(len(grad_year))} ({round(len(grad_year)/len(found_students)*100, 2)}%) records have graduation year discrepancies
{"{:,}".format(len(status_update))} ({round(len(status_update)/len(found_students)*100, 2)}%) records have status discrepancies, but correct graduation year

----

{"{:,}".format(len(manual))} status discrepancies flagged for manual review
{"{:,}".format(len(auto))} status discrepancies automatically processed
{"{:,}".format(len(unprocessed))} status discrepancies not processed
{"{:,}".format(len(new))} record updates since last scrape
'''
)


90,871 student records in AAMC-SRS scrape
88,310 (97.18%) records matched to AIMS on aamc_id
1,797 (1.98%) records found via matching process 
911 (1.0%) MEs appended via matching process
1,564 (1.72%) scraped records missing from our database

----

Of the records that matched on aamc_id
19,810 (22.36%) records are student affiliates
68,500 (77.3%) records are not student affiliates
    
----
Of the records that are not student affiliates
5,012 (7.32%) records have graduation year discrepancies
4,762 (6.95%) records have status discrepancies, but correct graduation year

----

2,750 status discrepancies flagged for manual review
1,422 status discrepancies automatically processed
590 status discrepancies not processed
1,046 record updates since last scrape



In [405]:
len(missing)

10125

In [ ]:
93,961 student records in AAMC-SRS scrape
91,435 (97.31%) records matched to AIMS on aamc_id
1,917 (2.04%) MEs appended via matching process
609 (0.65%) scraped records missing from our database

----

Of the records that matched on aamc_id
19,764 (21.62%) records are student affiliates
71,684 (78.4%) records are not student affiliates
    
----
Of the records that are not student affiliates
4,542 (6.34%) records have graduation year discrepancies
4,810 (6.71%) records have status discrepancies, but correct graduation year

----

4,660 status discrepancies flagged for manual review
4,692 records automatically processed
1,046 record updates since last scrape

In [63]:
pd.read_excel('../../Data/SRS/US_Schools_Crosswalk_AMA_AAMC.xlsx')

,INST_ID,INST_TYPE_CD,INST_NAME_120,AMA_ID
0,848,MSUS,Cooper Medical School of Rowan University,3307
1,854,MSUS,Charles E. Schmidt College of Medicine at Flor...,1108
2,867,MSUS,TCU and UNTHSC School of Medicine,4818
3,872,MSUS,New York University Long Island School of Medi...,3530
4,850,MSUS,University of South Carolina School of Medicin...,4505
...,...,...,...,...
182,844,MSUS,Oakland University William Beaumont School of ...,2517
183,851,MSUS,University of Arizona College of Medicine - Ph...,302
184,846,MSUS,Geisinger Commonwealth School of Medicine,4116
185,845,MSUS,Donald and Barbara Zucker School of Medicine a...,3550


In [7]:
scraped['Curr Status Date'] = pd.to_datetime(scraped['Curr Status Date'])
scraped_new['Curr Status Date'] = pd.to_datetime(scraped_new['Curr Status Date'])

In [8]:
scraped.columns

Index(['AAMC ID', 'Birth Country', 'Birth County', 'Birth State',
       'Citizen Country', 'Curr Class Level', 'Curr Class Level Eff Date',
       'Curr Exp Grad Date', 'Curr Med School Campus', 'Curr Status',
       ...
       'Unnamed: 45', 'Unnamed: 49', 'Unnamed: 53', 'Unnamed: 57',
       'Unnamed: 61', 'Unnamed: 65', 'Unnamed: 69', 'Unnamed: 73',
       'Unnamed: 85', 'Visa Desc'],
      dtype='object', length=120)

In [17]:
all_scraped = pd.merge(scraped, scraped_new, on='AAMC ID', suffixes = ['_old', ''], how = 'right')

In [19]:
all_scraped['Time_Diff'] = all_scraped['Curr Status Date'] - all_scraped['Curr Status Date_old']

In [22]:
new = all_scraped[all_scraped['Time_Diff']!='0 days'][scraped.columns]

In [385]:
new

,AAMC ID,Birth Country,Birth County,Birth State,Citizen Country,Curr Class Level,Curr Class Level Eff Date,Curr Exp Grad Date,Curr Med School Campus,Curr Status,...,Unnamed: 45,Unnamed: 49,Unnamed: 53,Unnamed: 57,Unnamed: 61,Unnamed: 65,Unnamed: 69,Unnamed: 73,Unnamed: 85,Visa Desc
388,14791954,United States of America (the),Nassau,New York,United States of America (the),2,07/20/2020,04/01/2024,NaN,Leave of Absence - Other Reasons,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1427,13570277,United States of America (the),Lubbock,Texas,United States of America (the),4,07/01/2019,06/01/2021,NaN,"No Longer in Research, Special Studies, or Joi...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1694,14499086,China,NaN,Foreign,China,2,08/03/2020,06/01/2024,NaN,Participating in Special Studies/Research (Non...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Permanent Resident
1696,13813663,China,NaN,Foreign,United States of America (the),4,07/01/2019,06/01/2021,NaN,"No Longer in Research, Special Studies, or Joi...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1902,14289441,United States of America (the),Cameron,Texas,United States of America (the),3,07/06/2020,06/01/2022,NaN,Leave of Absence - Health Reasons,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90866,15485622,United States of America (the),Jefferson,Louisiana,United States of America (the),2,01/01/2021,12/09/2022,U Washington  UW (Main Campus),Adv-Std From Dental,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90867,15485626,Canada,NaN,Ontario,United States of America (the),2,01/01/2021,12/09/2022,U Washington  UW (Main Campus),Adv-Std From Dental,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90868,15485645,United States of America (the),Becker,Minnesota,United States of America (the),2,01/01/2021,12/09/2022,U Washington  UW (Main Campus),Adv-Std From Dental,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90869,13516307,United States of America (the),Branch,Michigan,United States of America (the),3,04/05/2021,06/30/2023,NaN,Second time Enrollment,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [437]:
me_matched = pd.merge(matched, me_entity, on='entity_id')

In [389]:
me_matched

,Birth Country,Birth County,Birth State,Citizen Country,Curr Class Level,Curr Class Level Eff Date,Curr Exp Grad Date,Curr Med School Campus,Curr Status,Curr Status AcadYear,...,first_middle_x,first_middle_y,first_nm,gender,grad_yr,last_nm,middle_nm,school_id,stud_id,me
0,United States of America (the),Franklin,Kansas,United States of America (the),1,09/11/2006,05/12/2023,Arizona-Tucson (Main Campus),Second time Enrollment,2019,...,NaN,NaN,Khalid,M,2010,Jawadi,,14795400,39088708,01601101285
1,United States of America (the),Pima,Arizona,United States of America (the),2,08/13/2012,05/13/2022,Arizona-Tucson (Main Campus),"Completed Spec Stud, Resch or Jnt Degree",2019,...,NaN,NaN,Mitchell,M,2017,Bartlett,Julian,14794750,44676381,00301150725
2,United States of America (the),Broome,New York,United States of America (the),3,07/08/2019,05/20/2022,Texas Tech - Odessa,Returned from LOA,2019,...,NaN,NaN,Diana,F,2017,Hong,,14797103,46169418,03506171783
3,United States of America (the),Allegheny,Pennsylvania,United States of America (the),3,06/17/2014,05/08/2022,Illinois-Chicago (Main Campus),Second time Enrollment,2018,...,NaN,NaN,Thomas,M,2018,Hatzilabrou,,14795504,44003397,01611141609
4,United States of America (the),Travis,Texas,United States of America (the),4,02/10/2020,05/01/2021,NaN,Returned from LOA,2018,...,NaN,NaN,Jordan,M,2018,Price,Colovas,14798273,46907220,04814180706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7487,Cuba,NaN,Foreign,United States of America (the),1,08/01/2020,06/01/2028,NaN,Matriculated,2020,...,NaN,NaN,Joel,M,2028,Pardo,,14796505,63041290,02604281019
7488,Turkey,NaN,Foreign,Turkey,1,08/01/2020,06/01/2028,NaN,Matriculated,2020,...,NaN,NaN,Guldamla,F,2028,Kalender,,14796505,63018540,02604280993
7489,United States of America (the),Burlington,New Jersey,United States of America (the),1,08/27/2020,05/19/2028,MU South Carolina-Charleston (Main Campus),Matriculated,2020,...,NaN,NaN,Konnor,M,2028,Mcdowell,Patrick,14797948,63065080,04501280394
7490,United States of America (the),Ada,Idaho,United States of America (the),1,08/01/2020,06/01/2028,NaN,Matriculated,2020,...,NaN,NaN,Jacob,M,2028,Noeker,Alan,14796505,63030305,02604281001


In [328]:
for col in matched.columns:
    print(col)

Birth Country
Birth County
Birth State
Citizen Country
Curr Class Level
Curr Class Level Eff Date
Curr Exp Grad Date
Curr Med School Campus
Curr Status
Curr Status AcadYear
Curr Status Date
Current Degree Program
Date of Birth
Email
INST_ID
Legal Country
Legal County
Legal State
Matriculation Degree Program
Name
Preferred USMLE ID
Prev Campus Class Level 1
Prev Campus Class Level 2
Prev Campus Class Level 3
Prev Campus Class Level 4
Prev Exp Grad Date1
Prev Exp Grad Date10
Prev Exp Grad Date11
Prev Exp Grad Date12
Prev Exp Grad Date13
Prev Exp Grad Date14
Prev Exp Grad Date15
Prev Exp Grad Date16
Prev Exp Grad Date17
Prev Exp Grad Date18
Prev Exp Grad Date19
Prev Exp Grad Date2
Prev Exp Grad Date3
Prev Exp Grad Date4
Prev Exp Grad Date5
Prev Exp Grad Date6
Prev Exp Grad Date7
Prev Exp Grad Date8
Prev Exp Grad Date9
Prev Status AcadYear1
Prev Status AcadYear10
Prev Status AcadYear11
Prev Status AcadYear12
Prev Status AcadYear13
Prev Status AcadYear14
Prev Status AcadYear15
Prev Status A

In [473]:
me_matched[['me', 'Date of Birth',
'birth_dt',
'Name',
'first_nm',
'middle_nm',
'last_nm',
'gender',
'Sex', 'Birth State', 'birth_state_cd',
]]

,me,Date of Birth,birth_dt,Name,first_nm,middle_nm,last_nm,gender,Sex,Birth State,birth_state_cd
0,04706021651,12/30/1974,1974-12-30,"Manning, Christopher Brian",Christopher,Brian,Manning,M,M,Tennessee,NaN
1,04802210201,12/10/1990,1990-12-10,"Yarmoska, Steven Karl",Steven,Karl,Yarmoska,M,M,New Jersey,NaN
2,02401182066,09/25/1992,1992-09-25,"Patel, Nisarg Amrish",Nisarg,A,Patel,M,M,Arizona,NaN
3,02405200093,05/27/1988,1988-05-27,"Kowalski, Marek Mateusz",Marek,Mateusz,Kowalski,M,M,Foreign,NaN
4,02416200532,06/08/1993,1993-06-08,"Suzor, Matthew",Matthew,A,Suzor,M,M,Massachusetts,NaN
...,...,...,...,...,...,...,...,...,...,...,...
906,04601230599,12/21/1995,1995-12-21,"Van Oosbree, Annika Marie",Annika,Marie,Van Oosbree,F,F,Texas,NaN
907,00801221220,04/26/1995,1995-04-26,"Diab, Nicholas Sami",Nicholas,Sami,Diab,M,M,Texas,NaN
908,03806192294,12/03/1997,1997-12-03,"Bomani, Nichola Zakiya Bilal",Nichola,Zakiya Bilal,Bomani,F,F,Ohio,NaN
909,00801000657,07/17/1969,1969-07-17,"Kim, John Ihn",John,Ihn,Kim,M,M,Illinois,IL


In [477]:
matched = matched[['me','aamc_id',
'entity_id',
'school_id', 'category_code',
'stud_id', 'Date of Birth',
'birth_dt',
'Name',
'first_nm',
'middle_nm',
'last_nm',
'gender',
'Sex', 'Birth State', 'birth_state_cd',
'grad_yr',
'degree_cd',
'Current Degree Program',
'Curr Class Level',
'Curr Exp Grad Date',
'Curr Med School Campus',
'Curr Status',
'Curr Status AcadYear',
'Curr Status Date'
]]

In [476]:
matched = pd.merge(matched, me_entity, on='entity_id', how='left')

In [ ]:
# scraped
#found
# matched
# missing
# found_students
# stu_affil
# grad_year
# status_update
# manual
# auto
# new
f'''
{"{:,}".format(len(scraped))} student records in AAMC-SRS scrape
{"{:,}".format(len(found))} ({round(len(found)/len(scraped)*100, 2)}%) records matched to AIMS on aamc_id
{"{:,}".format(len(matched))} ({round(len(matched)/len(scraped)*100, 2)}%) records found via matching process 
{"{:,}".format(len(me_matched))} ({round(len(me_matched)/len(scraped)*100, 2)}%) MEs appended via matching process
{"{:,}".format(len(still_missing))} ({round(len(still_missing)/len(scraped)*100, 2)}%) scraped records missing from our database

----

Of the records that matched on aamc_id
{"{:,}".format(len(stu_affil))} ({round(len(stu_affil)/len(found)*100, 2)}%) records are student affiliates
{"{:,}".format(len(found_students))} ({round(len(found_students)/len(found)*100, 2)}%) records are not student affiliates
    
----
Of the records that are not student affiliates
{"{:,}".format(len(grad_year))} ({round(len(grad_year)/len(found_students)*100, 2)}%) records have graduation year discrepancies
{"{:,}".format(len(status_update))} ({round(len(status_update)/len(found_students)*100, 2)}%) records have status discrepancies, but correct graduation year

----

{"{:,}".format(len(manual) + len(all_other_sts))} status discrepancies flagged for manual review
{"{:,}".format(len(auto) + len(other_grad))} records automatically processed
{"{:,}".format(len(new))} record updates since last scrape
'''

In [414]:
me_matched.columns

Index(['Birth Country', 'Birth County', 'Birth State', 'Citizen Country',
       'Curr Class Level', 'Curr Class Level Eff Date', 'Curr Exp Grad Date',
       'Curr Med School Campus', 'Curr Status', 'Curr Status AcadYear',
       ...
       'first_middle_x', 'first_middle_y', 'first_nm', 'gender', 'grad_yr',
       'last_nm', 'middle_nm', 'school_id', 'stud_id', 'me'],
      dtype='object', length=145)

In [409]:
matched.columns

Index(['Birth Country', 'Birth County', 'Birth State', 'Citizen Country',
       'Curr Class Level', 'Curr Class Level Eff Date', 'Curr Exp Grad Date',
       'Curr Med School Campus', 'Curr Status', 'Curr Status AcadYear',
       ...
       'first_middle', 'first_middle_x', 'first_middle_y', 'first_nm',
       'gender', 'grad_yr', 'last_nm', 'middle_nm', 'school_id', 'stud_id'],
      dtype='object', length=144)

In [ ]:
f'''
{"{:,}".format(len(scraped_new))} student records in AAMC-SRS scrape
{"{:,}".format(len(found))} ({round(len(found)/len(scraped_new)*100, 2)}%) records matched to AIMS on aamc_id
{"{:,}".format(len(matched))} ({round(len(matched)/len(scraped_new)*100, 2)}%) records found via matching process 
{"{:,}".format(len(me_matched))} ({round(len(me_matched)/len(scraped_new)*100, 2)}%) MEs appended via matching process
{"{:,}".format(len(still_missing))} ({round(len(still_missing)/len(scraped_new)*100, 2)}%) scraped records missing from our database

----

Of the records that matched on aamc_id
{"{:,}".format(len(stu_affil))} ({round(len(stu_affil)/len(found)*100, 2)}%) records are student affiliates
{"{:,}".format(len(found_students))} ({round(len(found_students)/len(found)*100, 2)}%) records are not student affiliates
    
----
Of the records that are not student affiliates
{"{:,}".format(len(grad_year))} ({round(len(grad_year)/len(found_students)*100, 2)}%) records have graduation year discrepancies
{"{:,}".format(len(status_update))} ({round(len(status_update)/len(found_students)*100, 2)}%) records have status discrepancies, but correct graduation year

----

{"{:,}".format(len(manual) + len(all_other_sts))} status discrepancies flagged for manual review
{"{:,}".format(len(auto) + len(other_grad))} records automatically processed
{"{:,}".format(len(new))} record updates since last scrape
''')

In [ ]:
matched
missing
stu_affil
grad_year
status_discrepancy
new


In [424]:
grad_year[grad_year.aamc_id.isin(manual.aamc_id)]

,aamc_id,stud_id,grad_yr,CURRENT_GRAD_YEAR,school_id,edu_sts,edu_pref_cd,degree_cd,STATUS_CD,STATUS_DESC,...,INST_ID,Legal Country,Legal County,Legal State,Matriculation Degree Program,Name,Preferred USMLE ID,RaceEthnicity,Sex,Visa Desc
10,13909894,49244919,2021,2022,14797077,ENR,NON,MD,2.0,Matriculated,...,102,United States of America (the),Dakota,Minnesota,MD Program,"Blaeser, Anna Marie",5.4195e+07,White,F,None
12,14678581,51550847,2022,2023,14797077,ENR,NON,MD,2.0,Matriculated,...,102,United States of America (the),Bergen,New Jersey,Combined BS/MD,"Dabaghian, Antranik B",5.4389e+07,Unknown,M,None
16,14679714,51342171,2022,2023,14797077,ENR,NON,MD,2.0,Matriculated,...,102,United States of America (the),Rensselaer,New York,Combined BS/MD,"Gupta, Megan",5.43891e+07,Asian,F,None
17,14430891,49377779,2021,2022,14797077,ENR,NON,MD,2.0,Matriculated,...,102,United States of America (the),Virginia Beach City,Virginia,Combined BA/MD,"Hsu, Sharon Yi-Chang",5.41961e+07,Asian,F,None
20,14333679,49373229,2021,2022,14797077,ENR,NON,MD,2.0,Matriculated,...,102,United States of America (the),Warren,Ohio,Combined BS/MD,"Kumar, Aman",5.41958e+07,Asian,M,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4983,14327586,56614152,2023,2027,14795127,ENR,NON,MD,2.0,Matriculated,...,191,United States of America (the),Mercer,New Jersey,Combined MD/PhD,"Vetick, Michael Brandon",5.47679e+07,White,M,None
4986,14320409,56461948,2023,2024,43636173,ENR,NON,MD,2.0,Matriculated,...,845,United States of America (the),Suffolk,New York,MD Program,"Allen, Deshaun",5.46397e+07,"Black or African American; Hispanic, Latino, o...",M,None
4995,13385297,49180426,2021,2022,43636173,ENR,NON,MD,2.0,Matriculated,...,845,United States of America (the),Queens,New York,MD Program,"Lema, Maribel",5.41588e+07,"Hispanic, Latino, or of Spanish origin",F,None
5002,14276227,51366364,2022,2023,43636173,ENR,NON,MD,2.0,Matriculated,...,845,United States of America (the),PA UNK CNTY,Pennsylvania,MD Program,"Sahadeo, Tiffany Joy",5.4397e+07,Black or African American,F,None


In [425]:
grad_year

,aamc_id,stud_id,grad_yr,CURRENT_GRAD_YEAR,school_id,edu_sts,edu_pref_cd,degree_cd,STATUS_CD,STATUS_DESC,...,INST_ID,Legal Country,Legal County,Legal State,Matriculation Degree Program,Name,Preferred USMLE ID,RaceEthnicity,Sex,Visa Desc
0,13830010,49220401,2021,2022,14794737,ENR,NON,MD,1.0,Returned from LOA,...,816,United States of America (the),Harrison,Mississippi,MD Program,"Arshad, Arslan",5.42226e+07,Asian,M,None
1,13830010,49220401,2021,2022,14794737,ENR,NON,MD,1.0,Returned from LOA,...,816,United States of America (the),Harrison,Mississippi,MD Program,"Arshad, Arslan",5.42226e+07,Asian,M,None
2,14255874,49349153,2021,2022,14794737,ENR,NON,MD,1.0,Returned from LOA,...,816,United States of America (the),Etowah,Alabama,MD Program,"Hossain, Shyla R",5.42227e+07,Asian; White,F,None
3,14255874,49349153,2021,2022,14794737,ENR,NON,MD,1.0,Returned from LOA,...,816,United States of America (the),Etowah,Alabama,MD Program,"Hossain, Shyla R",5.42227e+07,Asian; White,F,None
4,13891294,49240694,2021,2022,14794737,LOA,NON,MD,1.0,Returned from LOA,...,816,United States of America (the),Etowah,Alabama,MD Program,"Kwan, Albert Kryan",5.42224e+07,Asian,M,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5007,13903798,49243580,2023,None,14795140,ENR,NON,MD,21.0,Transfer Out To U.S. Medical School,...,190,United States of America (the),Beaver,Pennsylvania,Combined MD/PhD,"Skaf, Yara Noor",5.43549e+07,Other,F,None
5008,13494458,47057799,2022,None,14797272,LOA,NON,MD,21.0,Transfer Out To U.S. Medical School,...,805,United States of America (the),Nassau,New York,MD Program,"Pinkhasov, Tzvia",5.36657e+07,White,F,None
5009,13968976,48330681,2021,None,14794815,ENR,NON,MD,21.0,Transfer Out To U.S. Medical School,...,822,United States of America (the),Alameda,California,MD Program,"Hufstedler, Shirley Lemus",5.413e+07,"American Indian or Alaska Native; Hispanic, La...",F,None
5010,13373796,47672075,2021,None,40624008,LOA,NON,MD,21.0,Transfer Out To U.S. Medical School,...,836,United States of America (the),Durham,North Carolina,MD Program,"Kim-Wang, Sophia Yoona",5.37429e+07,Asian,F,None


In [479]:
still_missing

,aamc_id,Birth Country,Birth County,Birth State,Citizen Country,Curr Class Level,Curr Class Level Eff Date,Curr Exp Grad Date,Curr Med School Campus,Curr Status,...,Unnamed: 57,Unnamed: 61,Unnamed: 65,Unnamed: 69,Unnamed: 85,Visa Desc,last_nm,first_middle,birth_state_cd,birth
37,13882491,United States of America (the),Mobile,Alabama,United States of America (the),2,08/03/2020,05/15/2023,NaN,Matriculated,...,NaN,NaN,NaN,NaN,NaN,NaN,Chambers II,Fredrick,AL,1992-01-27
38,14945687,United States of America (the),Lauderdale,Alabama,United States of America (the),1,08/03/2020,05/04/2024,NaN,Matriculated,...,NaN,NaN,NaN,NaN,NaN,NaN,Chastain,Claire Reagan,AL,1998-01-27
49,14860800,United States of America (the),Jefferson,Alabama,United States of America (the),1,08/03/2020,05/04/2024,NaN,Matriculated,...,NaN,NaN,NaN,NaN,NaN,NaN,Crider,Kramer Scott,AL,1997-07-15
55,14658727,United States of America (the),Mobile,Alabama,United States of America (the),1,08/03/2020,05/04/2024,NaN,Matriculated,...,NaN,NaN,NaN,NaN,NaN,NaN,Dean,Meghan Ann,AL,1995-11-10
56,15054395,United States of America (the),Mobile,Alabama,United States of America (the),1,08/03/2020,05/04/2024,NaN,Matriculated,...,NaN,NaN,NaN,NaN,NaN,NaN,Dean,Tiara,AL,1998-08-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90273,14533898,United States of America (the),San Luis Obispo,California,United States of America (the),2,09/01/2020,05/01/2023,NaN,Matriculated,...,NaN,NaN,NaN,NaN,NaN,NaN,O'Marr,Jamieson Michael,CA,1996-03-12
90302,12969068,United States of America (the),Essex,New Jersey,United States of America (the),3,07/03/2017,05/31/2023,NaN,Participating in Joint Degree Program,...,NaN,NaN,NaN,NaN,NaN,NaN,Quijano,Elias,NJ,1990-02-17
90310,13211035,United States of America (the),Hartford,Connecticut,United States of America (the),4,06/22/2020,05/01/2021,NaN,"Completed Spec Stud, Resch or Jnt Degree",...,NaN,NaN,NaN,NaN,NaN,NaN,Reed,Erin Jeanne,CT,1989-08-11
90397,12846918,Canada,NaN,Unknown Canadian,Canada,4,06/22/2020,05/01/2021,NaN,Adv-Std From Foreign Medical School,...,NaN,NaN,NaN,NaN,NaN,Permanent Resident,Voruganti,Rishi Teja,Unknown Canadian,1988-11-22


In [485]:
for col in stu_affil.columns:
    print(col)

aamc_id
Birth Country
Birth County
Birth State
Citizen Country
Curr Class Level
Curr Class Level Eff Date
Curr Exp Grad Date
Curr Med School Campus
Curr Status
Curr Status AcadYear
Curr Status Date
Current Degree Program
Date of Birth
Email
INST_ID
Legal Country
Legal County
Legal State
Matriculation Degree Program
Name
Preferred USMLE ID
Prev Campus Class Level 1
Prev Campus Class Level 2
Prev Campus Class Level 3
Prev Campus Class Level 4
Prev Exp Grad Date1
Prev Exp Grad Date10
Prev Exp Grad Date11
Prev Exp Grad Date12
Prev Exp Grad Date13
Prev Exp Grad Date14
Prev Exp Grad Date15
Prev Exp Grad Date16
Prev Exp Grad Date17
Prev Exp Grad Date18
Prev Exp Grad Date19
Prev Exp Grad Date2
Prev Exp Grad Date3
Prev Exp Grad Date4
Prev Exp Grad Date5
Prev Exp Grad Date6
Prev Exp Grad Date7
Prev Exp Grad Date8
Prev Exp Grad Date9
Prev Status AcadYear1
Prev Status AcadYear10
Prev Status AcadYear11
Prev Status AcadYear12
Prev Status AcadYear13
Prev Status AcadYear14
Prev Status AcadYear15
Prev 

In [493]:
stu_affil = stu_affil[['category_code','aamc_id',
  'stud_id',
  'grad_yr',
'school_id',
'edu_sts',
'edu_pref_cd',
'degree_cd',
'Birth Country',
'Birth County',
'Birth State',
'Citizen Country',
'Curr Class Level',
'Curr Class Level Eff Date',
'Curr Exp Grad Date',
'Curr Med School Campus',
'Curr Status',
'Curr Status AcadYear',
'Curr Status Date',
'Current Degree Program',
'Date of Birth',
'Email',
'INST_ID',
'Legal Country',
'Legal County',
'Legal State',
'Matriculation Degree Program',
'Name',
'Preferred USMLE ID',
'RaceEthnicity',
'Sex',
'Visa Desc']]

In [499]:
unprocessed = unprocessed[['aamc_id',
  'stud_id',
  'grad_yr',
'school_id',
'edu_sts',
'edu_pref_cd',
'degree_cd',
'Birth Country',
'Birth County',
'Birth State',
'Citizen Country',
'Curr Class Level',
'Curr Class Level Eff Date',
'Curr Exp Grad Date',
'Curr Med School Campus',
'Curr Status',
'Curr Status AcadYear',
'Curr Status Date',
'Current Degree Program',
'Date of Birth',
'Email',
'INST_ID',
'Legal Country',
'Legal County',
'Legal State',
'Matriculation Degree Program',
'Name',
'Preferred USMLE ID',
'RaceEthnicity',
'Sex',
'Visa Desc']]

In [507]:
new = new[['aamc_id',
'Birth Country',
'Birth County',
'Birth State',
'Citizen Country',
'Curr Class Level',
'Curr Class Level Eff Date',
'Curr Exp Grad Date',
'Curr Med School Campus',
'Curr Status',
'Curr Status AcadYear',
'Curr Status Date',
'Current Degree Program',
'Date of Birth',
'Email',
'INST_ID',
'Legal Country',
'Legal County',
'Legal State',
'Matriculation Degree Program',
'Name',
'Preferred USMLE ID',
'RaceEthnicity',
'Sex',
'Visa Desc']]

In [505]:
new['aamc_id']=new['AAMC ID']

In [513]:
news = new.aamc_id

In [529]:
scraped_new['NEW'] = ['Y' if x in list(new.aamc_id) else 'N' for x in scraped_new.aamc_id]

In [528]:
[x for x in scraped_new.aamc_id]

[13217475,
 13809625,
 14221239,
 14823457,
 15056929,
 15034284,
 14726249,
 14066807,
 14715551,
 14546434,
 13830010,
 14454144,
 14094979,
 14218721,
 14807906,
 14661977,
 14530441,
 14727705,
 14797569,
 14473971,
 15017334,
 14717445,
 14553236,
 14553494,
 14953934,
 14983759,
 14291251,
 14447675,
 14518300,
 14840639,
 14455842,
 14222028,
 14813259,
 13361170,
 14838024,
 15016898,
 14148616,
 13882491,
 14945687,
 14052249,
 14713191,
 13854338,
 14263804,
 13647163,
 14945749,
 14786388,
 14129800,
 14220207,
 14689499,
 14860800,
 14940621,
 14223972,
 14611066,
 13515614,
 14370235,
 14658727,
 15054395,
 13434294,
 14065468,
 14921590,
 14259688,
 14791816,
 14218530,
 14102719,
 13773757,
 14224226,
 14507989,
 14767855,
 14451564,
 15017423,
 14453162,
 14715261,
 14148380,
 14592337,
 14221696,
 14872702,
 14340742,
 14515995,
 14228974,
 14960601,
 14455267,
 14047727,
 14216104,
 14393182,
 13583126,
 14768905,
 12904116,
 13843172,
 14520224,
 14943038,
 14581398,

In [530]:
scraped_new[scraped_new.NEW=='Y']

,aamc_id,Birth Country,Birth County,Birth State,Citizen Country,Curr Class Level,Curr Class Level Eff Date,Curr Exp Grad Date,Curr Med School Campus,Curr Status,...,Unnamed: 45,Unnamed: 49,Unnamed: 53,Unnamed: 57,Unnamed: 61,Unnamed: 65,Unnamed: 69,Unnamed: 85,Visa Desc,NEW
271,14225620,United States of America (the),Cook,Illinois,United States of America (the),3,06/24/2019,05/09/2022,NaN,Leave of Absence - Other Reasons,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y
686,14791954,United States of America (the),Nassau,New York,United States of America (the),2,07/20/2020,04/01/2024,NaN,Leave of Absence - Other Reasons,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y
1028,13786919,United States of America (the),Salt Lake,Utah,United States of America (the),3,08/26/2019,05/01/2022,NaN,Participating in Special Studies/Research (Non...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y
1370,13786919,United States of America (the),Salt Lake,Utah,United States of America (the),3,08/26/2019,05/01/2022,NaN,Participating in Special Studies/Research (Non...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y
2408,13570277,United States of America (the),Lubbock,Texas,United States of America (the),4,07/01/2019,06/01/2021,NaN,"No Longer in Research, Special Studies, or Joi...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87747,14092553,United States of America (the),Santa Clara,California,United States of America (the),3,04/05/2021,06/01/2023,NaN,Returned from LOA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y
87768,14565713,United States of America (the),Vanderburgh,Indiana,United States of America (the),2,07/01/2020,06/01/2024,NaN,Leave of Absence - Other Reasons,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y
88478,14567407,United States of America (the),Ouachita,Louisiana,United States of America (the),1,07/29/2019,05/10/2024,NaN,Returned from LOA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y
88584,13874906,United States of America (the),Pima,Arizona,United States of America (the),3,04/30/2018,05/13/2022,NaN,Leave of Absence - Other Reasons,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y


In [520]:
new

,aamc_id,Birth Country,Birth County,Birth State,Citizen Country,Curr Class Level,Curr Class Level Eff Date,Curr Exp Grad Date,Curr Med School Campus,Curr Status,...,INST_ID,Legal Country,Legal County,Legal State,Matriculation Degree Program,Name,Preferred USMLE ID,RaceEthnicity,Sex,Visa Desc
388,14791954,United States of America (the),Nassau,New York,United States of America (the),2,07/20/2020,04/01/2024,NaN,Leave of Absence - Other Reasons,...,102,United States of America (the),Nassau,New York,MD Program,"Petrossian, Gregory Nicholas",54686480.0,White,M,NaN
1427,13570277,United States of America (the),Lubbock,Texas,United States of America (the),4,07/01/2019,06/01/2021,NaN,"No Longer in Research, Special Studies, or Joi...",...,104,United States of America (the),Denton,Texas,MD Program,"Cooney, Ashley Ann",53925624.0,White,F,NaN
1694,14499086,China,NaN,Foreign,China,2,08/03/2020,06/01/2024,NaN,Participating in Special Studies/Research (Non...,...,104,United States of America (the),Fort Bend,Texas,MD Program,"Li, Qiuyu",54648324.0,Asian,F,Permanent Resident
1696,13813663,China,NaN,Foreign,United States of America (the),4,07/01/2019,06/01/2021,NaN,"No Longer in Research, Special Studies, or Joi...",...,104,United States of America (the),Harris,Texas,MD Program,"Liang, Cynthia Shuang",53927182.0,Asian,F,NaN
1902,14289441,United States of America (the),Cameron,Texas,United States of America (the),3,07/06/2020,06/01/2022,NaN,Leave of Absence - Health Reasons,...,104,United States of America (the),Cameron,Texas,Combined MD/MPH,"Reddy, Rajadhar Tippi",54431267.0,Asian,M,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90866,15485622,United States of America (the),Jefferson,Louisiana,United States of America (the),2,01/01/2021,12/09/2022,U Washington  UW (Main Campus),Adv-Std From Dental,...,183,United States of America (the),St. Tammany,Louisiana,Combined MD/Dental (OMS),"Beckstrom, Thomas Oliver",NaN,White,M,NaN
90867,15485626,Canada,NaN,Ontario,United States of America (the),2,01/01/2021,12/09/2022,U Washington  UW (Main Campus),Adv-Std From Dental,...,183,United States of America (the),San Francisco,California,Combined MD/Dental (OMS),"Hui, Aaron",NaN,Asian,M,NaN
90868,15485645,United States of America (the),Becker,Minnesota,United States of America (the),2,01/01/2021,12/09/2022,U Washington  UW (Main Campus),Adv-Std From Dental,...,183,United States of America (the),Hennepin,Minnesota,Combined MD/Dental (OMS),"Smith, Charles",NaN,White,M,NaN
90869,13516307,United States of America (the),Branch,Michigan,United States of America (the),3,04/05/2021,06/30/2023,NaN,Second time Enrollment,...,185,United States of America (the),Branch,Michigan,MD Program,"Mohsen, Zaid",54243530.0,Other,M,NaN


In [524]:
scraped_new[scraped_new.aamc_id.isin(new.aamc_id)]

,aamc_id,Birth Country,Birth County,Birth State,Citizen Country,Curr Class Level,Curr Class Level Eff Date,Curr Exp Grad Date,Curr Med School Campus,Curr Status,...,Unnamed: 45,Unnamed: 49,Unnamed: 53,Unnamed: 57,Unnamed: 61,Unnamed: 65,Unnamed: 69,Unnamed: 85,Visa Desc,NEW
271,14225620,United States of America (the),Cook,Illinois,United States of America (the),3,06/24/2019,05/09/2022,NaN,Leave of Absence - Other Reasons,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
686,14791954,United States of America (the),Nassau,New York,United States of America (the),2,07/20/2020,04/01/2024,NaN,Leave of Absence - Other Reasons,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
1028,13786919,United States of America (the),Salt Lake,Utah,United States of America (the),3,08/26/2019,05/01/2022,NaN,Participating in Special Studies/Research (Non...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
1370,13786919,United States of America (the),Salt Lake,Utah,United States of America (the),3,08/26/2019,05/01/2022,NaN,Participating in Special Studies/Research (Non...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2408,13570277,United States of America (the),Lubbock,Texas,United States of America (the),4,07/01/2019,06/01/2021,NaN,"No Longer in Research, Special Studies, or Joi...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87747,14092553,United States of America (the),Santa Clara,California,United States of America (the),3,04/05/2021,06/01/2023,NaN,Returned from LOA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
87768,14565713,United States of America (the),Vanderburgh,Indiana,United States of America (the),2,07/01/2020,06/01/2024,NaN,Leave of Absence - Other Reasons,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
88478,14567407,United States of America (the),Ouachita,Louisiana,United States of America (the),1,07/29/2019,05/10/2024,NaN,Returned from LOA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
88584,13874906,United States of America (the),Pima,Arizona,United States of America (the),3,04/30/2018,05/13/2022,NaN,Leave of Absence - Other Reasons,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N


In [533]:
newly = scraped[['aamc_id','NEW']]

In [537]:
pd.merge(matched, newly, on='aamc_id').to_csv('../../Data/SRS/SRS_Matched_2021-03-19.csv', index=False)
pd.merge(still_missing, newly, on='aamc_id').to_csv('../../Data/SRS/SRS_Missing_2021-03-19.csv', index=False)
pd.merge(grad_year, newly, on='aamc_id').to_csv('../../Data/SRS/SRS_Graduation_Year_Discrepancy_2021-03-19.csv', index=False)
pd.merge(status_update, newly, on='aamc_id').to_csv('../../Data/SRS/SRS_Status_Discrepancy_2021-03-19.csv', index=False)
pd.merge(stu_affil, newly, on='aamc_id').to_csv('../../Data/SRS/SRS_Student_Affiliates_2021-03-19.csv', index=False)
pd.merge(auto, newly, on='aamc_id').to_csv('../../Data/SRS/SRS_Automatic_Updates_2021-03-19.csv', index=False)
pd.merge(manual, newly, on='aamc_id').to_csv('../../Data/SRS/SRS_Manual_Updates_2021-03-19.csv', index=False)
pd.merge(unprocessed, newly, on='aamc_id').to_csv('../../Data/SRS/SRS_Unprocessed_2021-03-19.csv', index=False)
new.to_csv('../../Data/SRS/SRS_New_2021-03-19.csv', index=False)